In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple
import gensim.downloader
from torch.optim import Adam
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import pickle
from torch import nn

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd
%cd /content/drive/MyDrive/Colab Notebooks/nlp2/nlptasks/notebooks/hroi-temp

/root
/content/drive/MyDrive/Colab Notebooks/nlp2/nlptasks/notebooks/hroi-temp


In [6]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 7.6 MB 42.0 MB/s 
     |████████████████████████████████| 163 kB 46.5 MB/s 


In [4]:
import pandas as pd
import numpy as np
#from datasets import load_dataset
#from transformers import AutoTokenizer
import math 
import torch

device = torch.device("cpu")
if torch.cuda.is_available():
 device = torch.device("cuda")
device

device(type='cpu')

In [6]:
#Load data
dataset = load_dataset("copenlu/answerable_tydiqa")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
})

In [7]:
#split data 
train_set = dataset["train"]
validation_set = dataset["validation"]

train_set = train_set.to_pandas()
validation_set = validation_set.to_pandas()

In [8]:
#preprocess data

eng_train_set = train_set[train_set['language'] == 'english']
#fin_train_set = train_set[train_set['language'] == 'finnish']
#jap_train_set = train_set[train_set['language'] == 'japanese']

eng_validation_set = validation_set[validation_set['language'] == 'english']
#fin_validation_set = validation_set[validation_set['language'] == 'finnish']
#jap_validation_set = validation_set[validation_set['language'] == 'japanese']

In [9]:
def get_answerability(annotations):
    answerable = []
    for annot in annotations:
        if -1 in annot['answer_start']:
            answerable.append(0)
        else:
            answerable.append(1)
    return answerable


eng_train_set['answerable'] = get_answerability(eng_train_set['annotations'].tolist())
#fin_train_set['answerable'] = get_answerability(fin_train_set['annotations'].tolist())
#jap_train_set['answerable'] = get_answerability(jap_train_set['annotations'].tolist())

eng_validation_set['answerable'] = get_answerability(eng_validation_set['annotations'].tolist())
#fin_validation_set['answerable'] = get_answerability(fin_validation_set['annotations'].tolist())
#jap_validation_set['answerable'] = get_answerability(jap_validation_set['annotations'].tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [12]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, T5Tokenizer, GPT2LMHeadModel, GPT2Tokenizer, GPT2Model
from transformers import AutoTokenizer
#%pip install SentencePiece 

eng_model_checkpoint = 'distilgpt2'
#jap_model_checkpoint = "rinna/japanese-gpt2-medium"
#fin_model_checkpoint = 'Finnish-NLP/gpt2-medium-finnish'

eng_tokenizer = AutoTokenizer.from_pretrained(eng_model_checkpoint)
#eng_model = AutoModelForCausalLM.from_pretrained(eng_model_checkpoint)
#model = model.to(device)

#jap_tokenizer = T5Tokenizer.from_pretrained(jap_model_checkpoint)
#jap_tokenizer.do_lower_case = True
#jap_model = AutoModelForCausalLM.from_pretrained(jap_model_checkpoint)


#fin_tokenizer = GPT2Tokenizer.from_pretrained(fin_model_checkpoint)
#fin_model = AutoModelForCausalLM.from_pretrained(fin_model_checkpoint)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += [questions[index] + "\n" + paragraphs[index]]
        
    return training_data

In [12]:
from datasets import DatasetDict, Dataset

def eng_tokenize_function(examples):
    return eng_tokenizer(examples["text"], truncation=True)

def fin_tokenize_function(examples):
    return fin_tokenizer(examples["text"], truncation=True)

def jap_tokenize_function(examples):
    return jap_tokenizer(examples["text"], truncation=True)

def make_df(train_set, validation_set):
  d_q = train_set['question_text'].tolist()
  d_p = train_set['document_plaintext'].tolist()
  training_data = question_parag_combine(d_q,d_p)
  training_labels = train_set['answerable'].tolist()

  d_q = validation_set['question_text'].tolist()
  d_p = validation_set['document_plaintext'].tolist()
  validation_data = question_parag_combine(d_q,d_p)
  validation_labels = validation_set['answerable'].tolist()



  data_set = {}
  sets = [['train',training_data, training_labels], ['val', validation_data, validation_labels]]
  for meta in sets:
      data_set[meta[0]] = {}
      data_set[meta[0]]['text'] = []
      data_set[meta[0]]['labels'] = []
      
      for ind, text in enumerate(meta[1]):
          data_set[meta[0]]['text'].append(text)
          data_set[meta[0]]['labels'].append(meta[2][ind])

          

          
  data_set = DatasetDict({'train':Dataset.from_dict(data_set['train']),
                          'valid':Dataset.from_dict(data_set['val'])\
                        })
  return data_set

eng_set = make_df(eng_train_set, eng_validation_set)
#fin_set = make_df(fin_train_set, fin_validation_set)
#jap_set = make_df(jap_train_set, jap_validation_set)

In [13]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    keys = ['attention_mask', 'input_ids']
    concatenated_examples = {k: sum(examples[k], []) for k in keys}
    total_length = len(concatenated_examples[list(keys)[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # this is needed as the used dataset is a subclass of ClassificationDataset, which requires label as a field...
    result["label"] = result["input_ids"].copy()
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
def tokenize_set(data_set, tokenize_function):
  tokenized_datasets = data_set.map(tokenize_function, batched=True, batch_size=1000, num_proc=4, remove_columns=["text"])
  tokenized_datasets_lm = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=4)
  tokenized_datasets_lm = tokenized_datasets_lm.remove_columns(['label'])
  return tokenized_datasets_lm

lm_eng = tokenize_set(eng_set, eng_tokenize_function)
#lm_fin = tokenize_set(fin_set, fin_tokenize_function)
#lm_jap = tokenize_set(jap_set, jap_tokenize_function)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
#model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    #max_steps=300
)

In [16]:
eng_trainer = Trainer(
    model=eng_model,
    args=training_args,
    train_dataset=lm_eng['train'],
    eval_dataset= lm_eng['valid'],
)
'''
fin_trainer = Trainer(
    model=fin_model,
    args=training_args,
    train_dataset=lm_fin['train'],
    eval_dataset= lm_fin['valid'],
)

jap_trainer = Trainer(
    model=jap_model,
    args=training_args,
    train_dataset=lm_jap['train'],
    eval_dataset= lm_jap['valid'],
)
'''

"\nfin_trainer = Trainer(\n    model=fin_model,\n    args=training_args,\n    train_dataset=lm_fin['train'],\n    eval_dataset= lm_fin['valid'],\n)\n\njap_trainer = Trainer(\n    model=jap_model,\n    args=training_args,\n    train_dataset=lm_jap['train'],\n    eval_dataset= lm_jap['valid'],\n)\n"

In [17]:
eng_eval_results = eng_trainer.evaluate()
#fin_eval_results = fin_trainer.evaluate()
#jap_eval_results = jap_trainer.evaluate()

print(f"Perplexity: {math.exp(eng_eval_results['eval_loss']):.2f}")
#print(f"Perplexity: {math.exp(fin_eval_results['eval_loss']):.2f}")
#print(f"Perplexity: {math.exp(jap_eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1107
  Batch size = 8


Perplexity: 48.74


In [18]:
eng_eval_results = eng_trainer.evaluate()
print(f"Perplexity: {math.exp(eng_eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1107
  Batch size = 8


Perplexity: 48.74


In [61]:
torch.save(eng_model, '../../models/eng_model_test.pth')

In [9]:
model = torch.load('../../models/eng_model_test.pth',map_location ='cpu')

In [14]:
# Export contextual representations:

def mean_pooling(model_output, attention_mask):
    # Mean Pooling - Take attention mask into account for correct averaging
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(model_output.size()).float()
    sum_embeddings = torch.sum(model_output * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


In [17]:
current_language = 'eng'
# import training data
dft = pd.read_csv('../../data/dft_'+current_language+'.csv')

# import validation data
#dfv = pd.read_csv('../../data/dfv_'+current_language+'.csv')

In [21]:
seed = 5555
size_train = 5000
size_valid = 2000

In [22]:
arange_dft = np.arange(len(dft))
np.random.seed(seed)
np.random.shuffle(arange_dft)

dft = dft.iloc[arange_dft]

train_df = dft.iloc[:size_train]
#valid_df = dft.iloc[size_train:size_train+size_valid]
#test_df = dfv

train_meta = train_df[['word_frequency_score','logres_pred']].values
#valid_meta = valid_df[['word_frequency_score','logres_pred']].values
#test_meta = test_df[['word_frequency_score','logres_pred']].values

In [25]:
# use eval to change string list to list get the length of each row
def create_batch(df, meta, model):
  combined = []
  que = df["question_text"].values
  text = df["document_plaintext"].values
  for index in range(len(df)):
    combined.append(str(que[index])+" "+str(text[index]))

  last_hidden = []
  for row in combined:
    encoded = eng_tokenizer(combined[0],
                        return_tensors='pt')
    encoded = {k:v.to(device) for k, v in encoded.items()}

    model_output = model(**encoded, output_hidden_states=True, return_dict=True)
    mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])
    
    last_hidden.append(mean_model_output)

  text_lens = [len(i) for i in combined] # length of all doc lists

  return combined, text_lens, df.label.values, meta
  

In [ ]:
train_dataset = create_batch(train_df, train_meta, model)
#test_dataset = create_batch(test_df.iloc[:100],test_meta[:100]) # rest of training for testing

In [45]:
combined = []
que = dfv["question_text"].values
text = dfv["document_plaintext"].values
for index in range(len(dfv)):
  combined.append(str(que[index])+" "+str(text[index]))

In [ ]:
train_data = []
valid_data = []
test_data = []
for row in combined:
  encoded = eng_tokenizer(combined[0],
                      return_tensors='pt')
  encoded = {k:v.to(device) for k, v in encoded.items()}

  model_output = eng_model(**encoded, output_hidden_states=True, return_dict=True)

  mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])

In [15]:
  encoded = eng_tokenizer('This is words',
                      return_tensors='pt')
  encoded = {k:v.to(device) for k, v in encoded.items()}

  model_output = model(**encoded, output_hidden_states=True, return_dict=True)

  mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])

In [16]:
mean_model_output

tensor([[-7.6618e-02,  3.0584e-01, -7.7772e-01,  2.7464e-01, -1.3996e-01,
          1.3528e-01,  2.2021e+00,  5.5615e-01, -1.6915e-01, -2.3814e-02,
          1.4020e-02,  4.3055e-02, -1.5713e-01,  1.1371e-01,  1.3940e-01,
          6.8808e-02, -2.5337e-01, -2.2795e-01,  3.8142e-02,  1.0883e-01,
          1.0344e-01, -1.0245e-01,  2.9589e-01, -3.3583e-01, -1.4743e-01,
         -9.0522e-02, -2.9030e-01, -2.0552e-01,  1.0589e-02,  1.2487e-01,
          2.2613e-02, -5.8727e-02, -7.3387e-02, -8.8789e-02, -6.8163e-03,
          3.8037e-01,  3.2271e+01,  2.7284e-01, -5.3693e-02,  5.0226e-01,
         -3.0682e-02,  2.8502e-01,  2.3457e-02,  2.2691e-02, -3.2705e-01,
         -8.4868e-02, -1.0843e-01, -2.5657e-01, -4.2096e-02,  5.6631e-01,
         -1.4963e-01,  3.5003e-01, -1.1763e-01, -5.4871e-03, -1.1295e-01,
          5.0442e-01, -5.6427e-04, -2.7593e-01,  5.1326e-02, -1.5914e-01,
          3.8852e-02,  1.5863e-01,  1.3620e-01, -1.2790e-01, -1.2219e+00,
         -8.4525e-02, -9.8851e-02, -2.

In [25]:
print(model_output.keys())
print(len(model_output['hidden_states'])) # contextual representations of separate words from each of the 6 layes
print(model_output['hidden_states'][-1].shape) # last layer with contextual representations (batch_size x num words x representation dim)

# Aggregate all the representations into one
mean_model_output2 = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])


odict_keys(['logits', 'past_key_values', 'hidden_states'])
7
torch.Size([1, 68, 768])


In [22]:
eng_tokenizer(eng_set['train']['text'][100],
                    return_tensors='pt')

{'input_ids': tensor([[ 2061,   389,   262,  9539,   286, 18522,    30,   198,   464,   509,
          9116, 43400,    12, 38328,  2746,   318,  2968,   306,  1900,   355,
           262,  1936,  9539,   286, 18522,    11,   996,   517, 14351,    11,
           262,  2746,  1281, 15968,   257, 17085,   286,  7016,  2585,  5924,
           416,  5651,   453,  2801,  3871,   706, 13669,    13,   383,  1936,
          9539,   389, 16199, 13437,    25, 14425,    11,  8993,    11, 23189,
            11,  8862,   290, 13427,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [1]:
import torch

In [2]:
test_dataset = torch.load('../../data/multi_test_batch')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.